In [1]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.component.map.base_map import BaseMap
from metadrive.component.map.pg_map import MapGenerateMethod
from metadrive.examples.ppo_expert.torch_expert import torch_expert as expert

import logging
from IPython.display import Image, clear_output
import pandas as pd
from pprint import pprint

from PIL import Image
import numpy as np

In [2]:
def create_env(seed=0):
    # ===== Termination Scheme =====
    termination_sceme = dict(
        out_of_route_done=False,
        on_continuous_line_done=False,
        crash_vehicle_done=True,
        crash_object_done=True,
        crash_human_done=True,
    )
    # ===== Map Config =====
    map_config = {
        BaseMap.GENERATE_TYPE: MapGenerateMethod.BIG_BLOCK_NUM,
        BaseMap.GENERATE_CONFIG: 5,  # 20 block
    }

    cfg = dict(
        # use_render=True,
        horizon=1000,  # figure out what do do aboutit
        record_episode=False,
        start_seed=seed,
        map_config=map_config,
        **termination_sceme
    )
    env = MetaDriveEnv(config=cfg)
    return env

In [3]:
def run_scenario(seed: int = 0):
    env = create_env(seed)
    _, step_info = env.reset(seed)

    while True:
        # get action from expert driving
        action = expert(env.agent, deterministic=True)
        obs, reward, tm, tr, step_info = env.step(action)

        if tm or tr:
            break

    # save_map_data(env)
    env.close()

In [4]:
run_scenario(10)

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): 1000
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 10, Num Scenarios : 1
[INFO] Use Torch PPO expert.
[INFO] Episode ended! Scenario Index: 10 Reason: arrive_dest.


In [5]:
env = create_env(seed=51)
obs, step_info = env.reset()

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): 1000
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 51, Num Scenarios : 1


In [6]:
for step in range(5):

    # get action from expert driving, or a dummy action
    action = expert(env.agent, deterministic=True)
    obs, reward, tm, tr, step_info = env.step(action)

    if tm or tr:
        break

In [30]:
def get_map_img(env):
    map = env.current_map.get_semantic_map(
        env.current_map.get_center_point(),
    )
    map = map.squeeze()  # reduce dimensionality
    map = (map * 255 * 4).astype(np.uint8)
    img = Image.fromarray(map)
    return img


get_map_img(env)

In [26]:
map = env.current_map
map_data = map.get_meta_data()
pprint(map_data["block_sequence"])

[{'id': 'I', 'pre_block_socket_index': None},
 {'change_lane_num': 0,
  'decrease_increase': 1,
  'id': 'T',
  'pre_block_socket_index': '0I-socket0',
  'radius': 10.0,
  't_type': 2},
 {'angle': 81.7476577758789,
  'dir': 0,
  'id': 'C',
  'length': 56.33229064941406,
  'pre_block_socket_index': '1T-socket0',
  'radius': 39.29075622558594},
 {'change_lane_num': 0,
  'decrease_increase': 1,
  'id': 'T',
  'pre_block_socket_index': '2C-socket0',
  'radius': 10.0,
  't_type': 1},
 {'id': 'S',
  'length': 75.14972686767578,
  'pre_block_socket_index': '3T-socket0'},
 {'angle': 60.0,
  'exit_radius': 7.76340389251709,
  'id': 'O',
  'inner_radius': 23.290212631225586,
  'pre_block_socket_index': '4S-socket0'}]


In [27]:
map.show_coordinates()